<a href="https://colab.research.google.com/github/detektor777/colab_list_video/blob/main/restore_white_balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive

In [ ]:
#@title ##**Select Video File** { display-mode: "form" }
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
from google.colab import drive

upload_option = "Load from Google Drive Root"  #@param ["Upload from PC", "Load from Google Drive Root", "Load from Google Drive"]

file_name = None
last_selected_button = None

def reset_button_colors(buttons):
    for btn in buttons:
        btn.style.button_color = None

if upload_option == "Upload from PC":
    print("Please upload a video file.")
    uploaded = files.upload()
    if uploaded:
        file_name = list(uploaded.keys())[0]
    else:
        print("No file uploaded.")
        file_name = None

elif upload_option == "Load from Google Drive Root":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    files_list = []

    for f in os.listdir(root_dir):
        if os.path.isfile(os.path.join(root_dir, f)) and os.path.splitext(f)[1].lower() in video_extensions:
            files_list.append(f)

    if not files_list:
        print("No video files found in Google Drive root.")
        file_name = None
    else:
        print("Select a video file from Google Drive root:")

        output = widgets.Output()
        buttons = []

        def on_button_clicked(b):
            global file_name, last_selected_button
            with output:
                clear_output()
                reset_button_colors(buttons)
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                else:
                    b.style.button_color = 'red'

                last_selected_button = b
                print(f"Selected file: {file_name if file_name else 'None'}")

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

elif upload_option == "Load from Google Drive":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    files_list = []

    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if os.path.splitext(f)[1].lower() in video_extensions:
                relative_path = os.path.relpath(os.path.join(dirpath, f), root_dir)
                files_list.append(relative_path)

    if not files_list:
        print("No video files found in Google Drive or its subfolders.")
        file_name = None
    else:
        print("Select a video file from Google Drive (including subfolders):")

        output = widgets.Output()
        buttons = []

        def on_button_clicked(b):
            global file_name, last_selected_button
            with output:
                clear_output()
                reset_button_colors(buttons)
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                else:
                    b.style.button_color = 'red'

                last_selected_button = b
                print(f"Selected file: {file_name if file_name else 'None'}")

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

if file_name:
    print(f"Video file path set to: {file_name}")
else:
    print("Video file path not set. Please select a file.")

In [ ]:
#@title ##**Config** { display-mode: "form" }
import os
from google.colab import files
import shutil
from google.colab import drive
output_folder = "google_drive" #@param ["google_drive","root"]

weight_white_patch = 0.3 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
priority_white_patch = "6" #@param ["0", "1", "2", "3", "4", "5", "6"]

weight_histogram = 0.2 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
priority_histogram = "1" #@param ["0", "1", "2", "3", "4", "5", "6"]

weight_gray_world = 0.0 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
priority_gray_world = "2" #@param ["0", "1", "2", "3", "4", "5", "6"]

weight_kmeans = 0.0 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
priority_kmeans = "3" #@param ["0", "1", "2", "3", "4", "5", "6"]

weight_color_temperature = 0.0 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
priority_color_temperature = "4" #@param ["0", "1", "2", "3", "4", "5", "6"]

weight_perfect_reflector = 0.3 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
priority_perfect_reflector = "5" #@param ["0", "1", "2", "3", "4", "5", "6"]

weight_skin_tone = 0.0 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
priority_skin_tone = "6" #@param ["0", "1", "2", "3", "4", "5", "6"]

upload_folder = 'upload'
result_folder = 'results'

if output_folder == "google_drive":
    if not os.path.exists('/content/drive'):
        print("Google Drive не подключён. Подключаем...")
        drive.mount('/content/drive')
    real_output_folder = '/content/drive/MyDrive/real_output'
    real_input_folder = "/content/drive/MyDrive/real_input"
elif output_folder == "root":
    real_output_folder = '/content/real_output'
    real_input_folder = "/content/real_input"

if not os.path.exists(real_output_folder):
    os.makedirs(real_output_folder)

if not os.path.exists(real_input_folder):
    os.makedirs(real_input_folder)

#clear folders
clear_input_folder = False #@param {type:"boolean"}
up_to_frame = "" #@param {type:"string"}
from_frame = "" #@param {type:"string"}

def clean_folder(folder_path, up_to=None, from_frame=None):
    print(f"\nCurrent parameters:")
    print(f"Delete frames up to: {up_to if up_to else 'not specified'}")
    print(f"Delete frames after: {from_frame if from_frame else 'not specified'}")

    if not os.path.isdir(folder_path):
        print(f"\nFolder {folder_path} does not exist!")
        print("Creating a new folder...")
        os.makedirs(folder_path)
        return

    if not up_to and not from_frame:
        print("\nNo parameters specified - deleting all folder content...")
        shutil.rmtree(folder_path)
        os.makedirs(folder_path)
        print(f"Folder {folder_path} cleared and recreated")
        return

    print("\nStarting file processing...")
    files = os.listdir(folder_path)
    jpg_files = [f for f in files if f.endswith('.jpg')]

    if not jpg_files:
        print("No JPG files to process in the folder")
        return

    deleted_count = 0
    processed_count = 0

    for filename in jpg_files:
        try:
            frame_number = int(filename.split('.')[0])
            should_delete = False

            if up_to and from_frame:
                if frame_number < int(up_to) or frame_number > int(from_frame):
                    should_delete = True
            elif up_to:
                if frame_number < int(up_to):
                    should_delete = True
            elif from_frame:
                if frame_number > int(from_frame):
                    should_delete = True

            if should_delete:
                file_path = os.path.join(folder_path, filename)
                os.remove(file_path)
                deleted_count += 1
                if deleted_count <= 5:
                    print(f'File deleted: {filename}')
                elif deleted_count == 6:
                    print('...')
            else:
                processed_count += 1

        except ValueError:
            print(f'Skipped file with invalid name: {filename}')

    print(f'\nProcessing complete:')
    print(f'Total files: {len(jpg_files)}')
    print(f'Files deleted: {deleted_count}')
    print(f'Files retained: {processed_count}')

if clear_input_folder:
    up_to_frame = up_to_frame if up_to_frame != "0" else None
    from_frame = from_frame if from_frame != "0" else None
    clean_folder(real_input_folder, up_to_frame, from_frame)

clear_output_folder = True #@param {type:"boolean"}

if clear_output_folder:
    if os.path.isdir(real_output_folder):
        shutil.rmtree(real_output_folder)
    os.makedirs(real_output_folder)

In [ ]:
#@title ##**Run sequence** { display-mode: "form" }
import cv2
import imageio
import os
import tqdm
import subprocess
import numpy as np
import time


library = "ffmpeg" #@param ["cv2","imageio","ffmpeg","skvideo","scipy","moviepy"]
delay = "0.1" #@param [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]


if (library == "ffmpeg"):
    !pip install ffmpeg-python
    import ffmpeg
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    probe = ffmpeg.probe(full_path)
    video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
    fps = video_info['r_frame_rate']
    duration = float(video_info['duration'])
    frame_count = int(video_info['nb_frames'])

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    pbar_ffmpeg = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    process = (
        ffmpeg
        .input(full_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', qscale=0)
        .run_async(pipe_stdout=True)
    )

    for i in range(frame_count):
        try:
            raw_video = process.stdout.read(video_info['width'] * video_info['height'] * 3)
            frame = np.frombuffer(raw_video, dtype='uint8').reshape((video_info['height'], video_info['width'], 3))
            frame_path = f"{real_input_folder}/{i:09d}.jpg"
            if os.path.isfile(frame_path):
              pbar_ffmpeg.update(1)
              continue
            imageio.imwrite(frame_path, frame)
        except Exception as e:
            print(f"Error writing to disk: {str(e)}. Retrying...")
            continue
        pbar_ffmpeg.update(1)
        time.sleep(float(delay))

    pbar_ffmpeg.close()
    process.wait()

#check
import os

def check_frames():
    frame_dir = real_input_folder
    frames = [int(f.split('.')[0].replace('frame', '')) for f in os.listdir(frame_dir) if f.endswith('.jpg')]
    min_frame = min(frames)
    max_frame = max(frames)
    print(min_frame)
    print(max_frame)

    missing_frames = []
    for i in range(min_frame, max_frame+1):
        if i not in frames:
            missing_frames.append(i)

    if len(missing_frames) > 0:
        print(f"Missing frames: {missing_frames}")
    else:
        print("All frames present")

attempts = 0
max_attempts = 10

while attempts < max_attempts:
    try:
        check_frames()
        break
    except Exception as e:
        attempts += 1
        print(f"Attempt {attempts} failed with error: {str(e)}")
        if attempts == max_attempts:
            print("Maximum attempts reached. Execution failed.")
        else:
            print("Retrying...")


In [ ]:
#@title ##**Run restore white balance** { display-mode: "form" }
import shutil
from tqdm import tqdm
import os
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

#check frames
def check_frames():
    frame_dir = real_input_folder
    frames = [int(f.split('.')[0].replace('frame', '')) for f in os.listdir(frame_dir) if f.endswith('.jpg')]
    min_frame = min(frames)
    max_frame = max(frames)
    print(min_frame)
    print(max_frame)

    missing_frames = []
    for i in range(min_frame, max_frame+1):
        if i not in frames:
            missing_frames.append(i)

    if len(missing_frames) > 0:
        print(f"Missing frames: {missing_frames}")
    else:
        print("All frames present")

attempts = 0
max_attempts = 10

while attempts < max_attempts:
    try:
        check_frames()
        break
    except Exception as e:
        attempts += 1
        print(f"Attempt {attempts} failed with error: {str(e)}")
        if attempts == max_attempts:
            print("Maximum attempts reached. Execution failed.")
        else:
            print("Retrying...")

#run
def white_patch_balance(img):
    b_max = np.max(img[:, :, 0])
    g_max = np.max(img[:, :, 1])
    r_max = np.max(img[:, :, 2])
    b_corrected = (img[:, :, 0] * (255.0 / b_max)).clip(0, 255).astype(np.uint8)
    g_corrected = (img[:, :, 1] * (255.0 / g_max)).clip(0, 255).astype(np.uint8)
    r_corrected = (img[:, :, 2] * (255.0 / r_max)).clip(0, 255).astype(np.uint8)
    return cv2.merge([b_corrected, g_corrected, r_corrected])

def histogram_balance(img):
    b, g, r = cv2.split(img)
    b_eq = cv2.equalizeHist(b)
    g_eq = cv2.equalizeHist(g)
    r_eq = cv2.equalizeHist(r)
    return cv2.merge([b_eq, g_eq, r_eq])

def gray_world_balance(img):
    b, g, r = cv2.split(img)
    b_mean = np.mean(b)
    g_mean = np.mean(g)
    r_mean = np.mean(r)
    mean = (b_mean + g_mean + r_mean) / 3
    b_scale = mean / b_mean if b_mean != 0 else 1
    g_scale = mean / g_mean if g_mean != 0 else 1
    r_scale = mean / r_mean if r_mean != 0 else 1
    b_corrected = (b * b_scale).clip(0, 255).astype(np.uint8)
    g_corrected = (g * g_scale).clip(0, 255).astype(np.uint8)
    r_corrected = (r * r_scale).clip(0, 255).astype(np.uint8)
    return cv2.merge([b_corrected, g_corrected, r_corrected])

def kmeans_balance(img, k=3):
    pixels = img.reshape((-1, 3)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = centers.astype(np.uint8)
    brightest_cluster = centers[np.argmax(np.sum(centers, axis=1))]
    scale = 255.0 / brightest_cluster
    corrected = (img * scale).clip(0, 255).astype(np.uint8)
    return corrected

def color_temperature_balance(img):
    b_mean = np.mean(img[:, :, 0])
    g_mean = np.mean(img[:, :, 1])
    r_mean = np.mean(img[:, :, 2])
    scale_r = g_mean / r_mean if r_mean != 0 else 1
    scale_b = g_mean / b_mean if b_mean != 0 else 1
    corrected = img.copy()
    corrected[:, :, 0] = (corrected[:, :, 0] * scale_b).clip(0, 255).astype(np.uint8)
    corrected[:, :, 2] = (corrected[:, :, 2] * scale_r).clip(0, 255).astype(np.uint8)
    return corrected

def perfect_reflector_balance(img):
    brightness = np.sum(img, axis=2)
    max_idx = np.unravel_index(np.argmax(brightness), brightness.shape)
    ref_color = img[max_idx]
    scale = 255.0 / ref_color
    corrected = (img * scale).clip(0, 255).astype(np.uint8)
    return corrected

def skin_tone_balance(img):
    ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycbcr)
    skin_mask = (cb >= 77) & (cb <= 127) & (cr >= 133) & (cr <= 173)
    if np.sum(skin_mask) == 0:
        return gray_world_balance(img)
    mean_cb = np.mean(cb[skin_mask])
    mean_cr = np.mean(cr[skin_mask])
    ideal_cb = 110
    ideal_cr = 140
    delta_cb = ideal_cb - mean_cb
    delta_cr = ideal_cr - mean_cr
    ycbcr_corrected = ycbcr.copy()
    ycbcr_corrected[:, :, 1] = (ycbcr[:, :, 1] + delta_cr).clip(0, 255).astype(np.uint8)
    ycbcr_corrected[:, :, 2] = (ycbcr[:, :, 2] + delta_cb).clip(0, 255).astype(np.uint8)
    corrected = cv2.cvtColor(ycbcr_corrected, cv2.COLOR_YCrCb2BGR)
    return corrected

algorithms = [
    ("White Patch", white_patch_balance, weight_white_patch, int(priority_white_patch)),
    ("Histogram", histogram_balance, weight_histogram, int(priority_histogram)),
    ("Gray World", gray_world_balance, weight_gray_world, int(priority_gray_world)),
    ("K-Means", kmeans_balance, weight_kmeans, int(priority_kmeans)),
    ("Color Temperature", color_temperature_balance, weight_color_temperature, int(priority_color_temperature)),
    ("Perfect Reflector", perfect_reflector_balance, weight_perfect_reflector, int(priority_perfect_reflector)),
    ("Skin Tone", skin_tone_balance, weight_skin_tone, int(priority_skin_tone))
]

algorithms_sorted = sorted(algorithms, key=lambda x: x[3])

upload_folder = "/content/upload"
result_folder = "/content/results"

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder)

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

file_list = os.listdir(real_input_folder)
file_list.sort()
frames = [int(f.split('.')[0].replace('', '')) for f in file_list if f.endswith('.jpg')]
min_frame = min(frames)

real_files = os.listdir(real_output_folder)
if real_files:
    real_frames = [int(re.findall(r'(\d+)_out\.jpg', f)[0]) for f in real_files if re.match(r'\d+_out\.jpg', f)]
    start_frame = max(real_frames) + 1
else:
    start_frame = min_frame

max_frame = frames[-1]
print(f"max frame: {max_frame}")
files_to_copy = [f"{real_input_folder}/{frame:09d}.jpg" for frame in range(start_frame, max_frame+1) if f"{frame:09d}.jpg" in file_list]

total_files = len(files_to_copy)
batch_size = 10
num_iterations = (total_files + batch_size - 1) // batch_size

print(f"start frame: {start_frame}")
print(f"min frame: {min_frame}")
print(f"total: {total_files}")
print(f"iterations: {num_iterations}")

with tqdm(total=num_iterations) as pbar:
    for i in range(0, total_files, batch_size):
        batch_files = files_to_copy[i:i+batch_size]

        for file in batch_files:
            copied = False
            while not copied:
                try:
                    shutil.copy(file, upload_folder)
                    copied = True
                except:
                    print(f"File {file} failed to copy. Retrying...")
                    continue

        for img_path in glob.glob(os.path.join(upload_folder, "*.jpg")):
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result_rgb = img_rgb.copy()

            for name, func, weight, priority in algorithms_sorted:
                if weight > 0:
                    corrected = func(img)
                    corrected_rgb = cv2.cvtColor(corrected, cv2.COLOR_BGR2RGB)
                    result_rgb = cv2.addWeighted(result_rgb, 1 - weight, corrected_rgb, weight, 0)

            output = cv2.cvtColor(result_rgb, cv2.COLOR_RGB2BGR)

            imgname = os.path.basename(img_path)  # Изменено: убрано .replace('.jpg', '_out.jpg')
            save_path = os.path.join(result_folder, imgname)
            cv2.imwrite(save_path, output)

        copied = False
        while not copied:
            try:
                shutil.copytree(result_folder, real_output_folder, dirs_exist_ok=True)
                copied = True
            except:
                print("Failed to copy result folder. Retrying...")
                continue

        if os.path.isdir(upload_folder):
            shutil.rmtree(upload_folder)
        os.makedirs(upload_folder)

        if os.path.isdir(result_folder):
            shutil.rmtree(result_folder)
        os.makedirs(result_folder)

        pbar.update(1)

total_upload_files = len(os.listdir('/content/upload'))
print(f"total upload files: {total_upload_files}")

#check frames
import os

def check_frames():
    frame_dir = real_output_folder
    frames = [int(f.split('.')[0].replace('frame', '')) for f in os.listdir(frame_dir) if f.endswith('.jpg')]
    min_frame = min(frames)
    max_frame = max(frames)
    print(min_frame)
    print(max_frame)

    missing_frames = []
    for i in range(min_frame, max_frame+1):
        if i not in frames:
            missing_frames.append(i)

    if len(missing_frames) > 0:
        print(f"Missing frames: {missing_frames}")
    else:
        print("All frames present")

attempts = 0
max_attempts = 10

while attempts < max_attempts:
    try:
        check_frames()
        break
    except Exception as e:
        attempts += 1
        print(f"Attempt {attempts} failed with error: {str(e)}")
        if attempts == max_attempts:
            print("Maximum attempts reached. Execution failed.")
        else:
            print("Retrying...")

In [ ]:
#@title ##**Create video** { display-mode: "form" }
import cv2
import os
import subprocess
from moviepy.editor import ImageSequenceClip, VideoFileClip
from tqdm.notebook import tqdm


print(f"output_folder: {output_folder}")

if 'file_name' in locals() and os.path.exists(file_name):
    base_file_name = os.path.basename(file_name)
else:
    raise ValueError("file_name is not defined or the file does not exist")

if output_folder == "google_drive":
    save_path = '/content/drive/MyDrive/'
elif output_folder == "root":
    save_path = '/content/'
else:
    save_path = '/content/'

full_path = os.path.join(save_path, base_file_name) if not os.path.exists(file_name) else file_name
output_file_name = base_file_name.rsplit('.', 1)[0] + '_white_balance.mp4'
output_file = os.path.join(save_path, output_file_name)
temp_video = "/content/temp_video.mp4"

cap = cv2.VideoCapture(full_path)
fps_of_video = int(cap.get(cv2.CAP_PROP_FPS))
cap.release()

img_files = [os.path.join(real_output_folder, img) for img in os.listdir(real_output_folder)]
img_files.sort()

if img_files:
    first_frame = cv2.imread(img_files[0])
    height, width = first_frame.shape[:2]
else:
    raise ValueError("No images found in the output folder")

def get_video_bitrate(file_path):
    cmd = ['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=bit_rate', '-of', 'default=noprint_wrappers=1:nokey=1', file_path]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    bitrate = result.stdout.strip()
    try:
        return int(bitrate)
    except ValueError:
        return None

bitrate = get_video_bitrate(full_path)
if bitrate:
    bitrate = int(bitrate * 1)
    bitrate_str = f'{bitrate // 1000}k'
else:
    bitrate_str = '7500k'

frames = []
for img_file in tqdm(img_files, desc="Processing frames"):
    frame = cv2.imread(img_file)
    if frame.shape[:2] != (height, width):
        frame = cv2.resize(frame, (width, height))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

clip = ImageSequenceClip(frames, fps=fps_of_video)
clip.write_videofile(temp_video, codec='libx264', bitrate=bitrate_str, audio=False)

cmd = ['ffmpeg', '-i', temp_video, '-i', full_path, '-map', '0:v', '-map', '1:a?', '-map', '1:s?', '-c', 'copy', '-y', output_file]
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if os.path.exists(output_file):
    if os.path.exists(temp_video):
        os.remove(temp_video)
    print("Video created successfully")
    print(f"Video saved at: {output_file}")
else:
    print("Failed to create video")
    print(f"Expected save path: {output_file}")
    print(f"FFmpeg error output: {result.stderr}")


In [ ]:
#@title ##**Compare videos side by side (optional)** { display-mode: "form" }
import os
from moviepy.editor import VideoFileClip, clips_array
from IPython.display import display, Video

if 'file_name' not in locals() or not os.path.exists(file_name):
    raise ValueError("file_name is not defined or the original video file does not exist")
if 'output_file' not in locals() or not os.path.exists(output_file):
    raise ValueError("output_file is not defined or the processed video file does not exist")

original_video_path = file_name
processed_video_path = output_file

original_clip = VideoFileClip(original_video_path)
processed_clip = VideoFileClip(processed_video_path)

min_duration = min(original_clip.duration, processed_clip.duration)
original_clip = original_clip.subclip(0, min_duration)
processed_clip = processed_clip.subclip(0, min_duration)

if original_clip.size != processed_clip.size:
    target_size = (max(original_clip.w, processed_clip.w), max(original_clip.h, processed_clip.h))
    original_clip = original_clip.resize(target_size)
    processed_clip = processed_clip.resize(target_size)

final_clip = clips_array([[original_clip, processed_clip]])

save_path = os.path.dirname(output_file)
comparison_file_name = os.path.basename(file_name).rsplit('.', 1)[0] + '_comparison.mp4'
comparison_file = os.path.join(save_path, comparison_file_name)

final_clip.write_videofile(comparison_file, codec='libx264', audio=True, fps=original_clip.fps)

if os.path.exists(comparison_file):
    print(f"Comparison video saved at: {comparison_file}")
    display(Video(comparison_file, embed=True, width=800))
else:
    print("Failed to create comparison video")

In [ ]:
#@title ##**Download video** { display-mode: "form" }
import cv2
from google.colab import files
import os
import time
from tqdm.notebook import tqdm

if output_folder == "google_drive":
    path = '/mnt/gdrive/MyDrive/'
elif output_folder == "root":
    path = '/content/'

output_file_name = file_name.rsplit('.', 1)[0] + '_upscale.mp4'
output_file = os.path.join(path, output_file_name)

if not os.path.exists(output_file):
    raise FileNotFoundError(f"Video file {output_file} not found. Make sure the video creation cell was executed successfully.")

with tqdm(total=1, desc="Downloading video") as pbar:
    files.download(output_file)

    while os.path.exists(output_file) and not os.path.getsize(output_file) == 0:
        time.sleep(0.5)
    pbar.update(1)

print("Download completed")
